In [1]:
import plotly
plotly.tools.set_credentials_file(username='yandixin_929', api_key='P63iFm9buoejxvCqeSQq')

In [2]:
import plotly.plotly as py   #for online plotting
from plotly.figure_factory import create_table
import plotly.graph_objs as go
from plotly.grid_objs import Grid, Column
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot  #for offline plotting
#init_notebook_mode(connected=True)  #for offline plotting

import pandas as pd
import time

In [3]:
file_dir = '/Users/yandixin_929/Desktop/MSAN/Data_Visualization/project/'
dataset = pd.read_csv(file_dir + 'agg_country_with_continent.csv')

In [4]:
# make a list of years
years = dataset.Year.unique()

In [5]:
# make list of continents
continents = dataset.Continent.unique()

### Countries with major surplus with US 

In [6]:
# Naive determination - sort by the 20-yr mean surplus/deficit in ascending color and select the top 15
deficit_countries = list(dataset.groupby('Country_name')['Surplus/Deficit'].mean().sort_values().reset_index().loc[0:15, 'Country_name'])

In [7]:
deficit_table = dataset.loc[(dataset['Country_name'].isin(deficit_countries)) & (dataset['Surplus/Deficit'] < 0), :]

deficit_table['Deficit'] = deficit_table['Surplus/Deficit'].abs()

deficit_table.describe()

,Year,Country_code,Surplus/Deficit,Imports,Exports,Deficit
count,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000
mean,2006.984894,4734.344411,-37419.051842,81665.751104,44246.699262,37419.051842
std,6.016373,1527.128455,57820.513756,99046.161539,64034.456712,57820.513756
min,1997.000000,1220.000000,-375227.535400,4176.006873,627.900000,341.600000
25%,2002.000000,4190.000000,-37425.806185,25228.416765,8992.800000,12315.150000
50%,2007.000000,5170.000000,-18743.576960,37408.943950,18485.640880,18743.576960
75%,2012.000000,5700.000000,-12315.150000,101148.300000,48539.805615,37425.806185
max,2017.000000,7530.000000,-341.600000,505597.065000,312816.950500,375227.535400


In [8]:
columns = []
# make grid
for year in years:
    for continent in continents:
        dataset_by_year = deficit_table[deficit_table['Year'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['Continent'] == continent]
        for col_name in dataset_by_year_and_cont:
            # each column name is unique
            column_name = '{year}_{continent}_{header}_surplus_grid'.format(
                year=year, continent=continent, header=col_name
            )
            a_column = Column(list(dataset_by_year_and_cont[col_name]), column_name)
            columns.append(a_column)

In [9]:
# upload grid
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'surplus_grid'+str(time.time()), auto_open=False)
url

'https://plot.ly/~yandixin_929/6/'

In [10]:
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [3.5, 6], 'title': 'Imports in million', 'zerolinewidth': 1,
        'ticklen': 5,
        'gridwidth': 2, 'gridcolor':'rgb(255, 255, 255)', 'type': 'log'}
figure['layout']['yaxis'] = {'range': [3.5, 6], 'title': 'Exports in million', 'zerolinewidth': 1,
        'ticklen': 5,
        'gridwidth': 2, 'gridcolor':'rgb(255, 255, 255)', 'type': 'log'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['paper_bgcolor'] = 'rgb(243, 243, 243)'
figure['layout']['plot_bgcolor'] = 'rgb(243, 243, 243)'
figure['layout']['title'] = 'Exports vs Imports for Countries with Major Surplus wrt US'

In [11]:
# make slider
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}


In [12]:
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1997',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

In [13]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

In [14]:
# new make data
col_name_template = '{year}_{continent}_{header}_surplus_grid'
year = 1997
for continent in continents:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Imports'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Exports'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Country_name'
        )),
        'marker': {
            'sizemode': 'area',
            'sizeref': 200,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, continent=continent, header='Deficit'))
        },
        'name': continent
    }
    figure['data'].append(data_dict)

In [15]:
# new make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for continent in continents:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Imports'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Exports'
        )),
            'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Country_name'
        )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 200,
                'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, continent=continent, header='Deficit'))
            },
            'name': continent
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

In [16]:
figure['layout']['sliders'] = [sliders_dict]

In [17]:
py.icreate_animations(figure, 'surplus'+str(time.time()))

### Countries with major deficit with US 

In [18]:
# Naive determination - sort by the 20-yr mean surplus/deficit in descending color and select the top 15
surplus_countries = list(dataset.groupby('Country_name')['Surplus/Deficit'].mean().sort_values(ascending = False).reset_index().loc[0:15, 'Country_name'])

In [19]:
surplus_table = dataset.loc[(dataset['Country_name'].isin(surplus_countries)) & (dataset['Surplus/Deficit'] > 0), :]
surplus_table['Surplus'] = surplus_table['Surplus/Deficit'].abs()
surplus_table.describe()

,Year,Country_code,Surplus/Deficit,Imports,Exports,Surplus
count,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2007.528239,4308.976744,6529.041880,6636.917860,13165.959741,6529.041880
std,6.121822,1515.644073,6847.375084,7070.143690,11763.634133,6847.375084
min,1997.000000,2250.000000,40.200000,142.400000,354.400000,40.200000
25%,2002.000000,2470.000000,1705.700000,764.809491,3334.100000,1705.700000
50%,2008.000000,4231.000000,3657.059932,4261.021154,9750.819527,3657.059932
75%,2013.000000,5590.000000,9795.498752,9806.700000,20005.427960,9795.498752
max,2017.000000,7290.000000,36678.144590,32123.378690,44105.506960,36678.144590


In [20]:
columns = []
# make grid
for year in years:
    for continent in continents:
        dataset_by_year = surplus_table[surplus_table['Year'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['Continent'] == continent]
        for col_name in dataset_by_year_and_cont:
            # each column name is unique
            column_name = '{year}_{continent}_{header}_deficit_grid'.format(
                year=year, continent=continent, header=col_name
            )
            a_column = Column(list(dataset_by_year_and_cont[col_name]), column_name)
            columns.append(a_column)

In [21]:
# upload grid
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'deficit_grid'+str(time.time()), auto_open=False)
url

'https://plot.ly/~yandixin_929/8/'

In [22]:
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [2.9, 4.6], 'title': 'Imports in million', 'zerolinewidth': 1,
        'ticklen': 5,
        'gridwidth': 2, 'gridcolor':'rgb(255, 255, 255)', 'type': 'log'}
figure['layout']['yaxis'] = {'range': [3.2, 5], 'title': 'Exports in million', 'zerolinewidth': 1,
        'ticklen': 5,
        'gridwidth': 2, 'gridcolor':'rgb(255, 255, 255)', 'type': 'log'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['paper_bgcolor'] = 'rgb(243, 243, 243)'
figure['layout']['plot_bgcolor'] = 'rgb(243, 243, 243)'
figure['layout']['title'] = 'Exports vs Imports for Countries with Major Deficit with US'

In [23]:
# make slider
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [24]:
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1997',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

In [25]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

In [26]:
# new make data
col_name_template = '{year}_{continent}_{header}_deficit_grid'
year = 1997
for continent in continents:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Imports'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Exports'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Country_name')),
        'marker': {
            'sizemode': 'area',
            'sizeref': 50,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, continent=continent, header='Surplus'))
        },
        'name': continent
    }
    figure['data'].append(data_dict)

In [27]:
# new make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for continent in continents:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Imports'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Exports'
        )),
            'mode': 'markers',
            'textsrc': grid.get_column_reference(col_name_template.format(
            year=year, continent=continent, header='Country_name'
        )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 50,
               'sizesrc': grid.get_column_reference(col_name_template.format(
                 year=year, continent=continent, header='Surplus'))
            },
            'name': continent
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

In [28]:
figure['layout']['sliders'] = [sliders_dict]

In [29]:
py.icreate_animations(figure, 'deficit'+str(time.time()))